# 네이버 매장정보 크롤링

### 네이버 매장 존재 여부 확인 크롤링에서 만든 파일을 사용합니다

문제 
1. 강남구 매장이지만, 네이버로 검색하는 경우 강남구가 아닌 다른 지역의 매장명이 검색됨
2. 데이터 반영이 빠른편이지만, 네이버에 매장이 없는 경우가 있음 (없는 매장 : 맥도날드, 교촌, 굽네) 
3. 검색하면 전혀 다른 업체가 나오는 경우가 있음 -> 네이버 업태구분명을 크롤링해와서 참고하여 반영해야 
4. 나라에서 제공하는 data를 사용하는 경우 인기도를 알기 어려움. 아닌가. -> 네이버 지도 api를 사용해서 점수를 부여할까? 
5. 자료 업데이트 시기 문제 - data를 다운로드 받아서 사용하는 경우, 매장 정보는 계속 바뀐는데 업데이트가 늦잖아... 매장 정보의 변동이 생기 사람이 계속 업로드를 해야 하잖아.. 

In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os

In [2]:
# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
# driver = webdriver.Chrome(ChromeDriverManager().install())


# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()


# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)

In [3]:
# 데이터 불러오기
original_res_df = pd.read_excel('combined_file.xlsx')
res_df = original_res_df.copy()
res_df = res_df.loc[res_df['검색여부'] == '가능'].reset_index(drop=True)
res_df.drop('Unnamed: 0', axis=1, inplace=True)


# 컬럼 순서 바꾸기 -> 보기 좋으라고 바꿈
res_df['검색어'] = res_df['상세주소'] + " " + res_df['사업장명']
column_order = ['검색어', '업태구분명', '사업장명', '사업장명_원본', '시도', '지역구', '상세주소', '소재지전체주소', '도로명전체주소', '좌표정보(x)', '좌표정보(y)', '검색여부', '영업상태명']
res_df = res_df[column_order]
res_df.tail()

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
8291,"도곡로 401, 간코",일식,간코,간코 롯데백화점강남점,서울특별시,강남구,"도곡로 401,",서울특별시 강남구 대치동 937 롯데백화점,"서울특별시 강남구 도곡로 401, 롯데백화점 지하1층 (대치동)",204669.543367,443873.621189,가능,영업/정상
8292,"압구정로 108, 오토김밥",분식,오토김밥,오토김밥 신사점,서울특별시,강남구,"압구정로 108,",서울특별시 강남구 신사동 528-3 덕산빌딩,"서울특별시 강남구 압구정로 108, 덕산빌딩 지상1층 10호 (신사동)",201749.456414,446742.828124,가능,영업/정상
8293,"압구정로42길 36, 마들마들",한식,마들마들,마들마들,서울특별시,강남구,"압구정로42길 36,",서울특별시 강남구 신사동 635-14 금광아크존1,"서울특별시 강남구 압구정로42길 36, 지상1층 101호 (신사동, 금광아크존1)",202990.870783,447146.555568,가능,영업/정상
8294,"언주로151길 21, 세바",경양식,세바,세바(Sevva),서울특별시,강남구,"언주로151길 21,",서울특별시 강남구 신사동 629-1,"서울특별시 강남구 언주로151길 21, 지상1층 (신사동)",202731.421127,446609.460134,가능,영업/정상
8295,"논현로161길 37, 구구구",경양식,구구구,구구구 (GOOGOOGOO),서울특별시,강남구,"논현로161길 37,",서울특별시 강남구 신사동 552-5,"서울특별시 강남구 논현로161길 37, 동쪽 지상1층 (신사동)",202153.410454,446748.562578,가능,영업/정상


In [4]:
res_info_df = pd.DataFrame()
restaurant_name_list = []
category_name_list = []
address_list = []
menu_list = []
price_list = []
review_list = [] 

# 여기서부턴 테스트 -> 밑에 찐코드 있습니다

In [5]:
# 검색할 데이터 수 지정 
sample_n = 30

# # 1. 인덱스 기준 200개 추출
# df_test = df.iloc[:sample_n]

# 2. 랜덤으로 100개 추출
df_test = res_df.sample(n=sample_n, random_state=42)
df_test.reset_index(inplace=True, drop=True)
df_test.head(10)

df_test

,검색어,업태구분명,사업장명,사업장명_원본,시도,지역구,상세주소,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),검색여부,영업상태명
0,"논현로85길 29, 남오토코",일식,남오토코,남오토코,서울특별시,강남구,"논현로85길 29,",서울특별시 강남구 역삼동 739-8,"서울특별시 강남구 논현로85길 29, 2층 201호 (역삼동)",203089.667994,444036.257391,가능,영업/정상
1,테헤란로 124 뢰벤 돈까스,한식,뢰벤 돈까스,뢰벤 돈까스,서울특별시,강남구,테헤란로 124,서울특별시 강남구 역삼동 823-0번지 지하1층B102호,"서울특별시 강남구 테헤란로 124 (역삼동,지하1층B102호)",202722.531831,444055.070696,가능,영업/정상
2,남부순환로 2738 생강김밥,분식,생강김밥,생강김밥,서울특별시,강남구,남부순환로 2738,서울특별시 강남구 도곡동 411-20번지 1층101호,"서울특별시 강남구 남부순환로 2738 (도곡동,1층101호)",203948.270103,442641.122188,가능,영업/정상
3,"학동로101길 7, 삼성양곱창",한식,삼성양곱창,삼성양곱창,서울특별시,강남구,"학동로101길 7,",서울특별시 강남구 청담동 133-6,"서울특별시 강남구 학동로101길 7, 지상1층 109호 (청담동)",205024.620584,446516.758471,가능,영업/정상
4,"테헤란로81길 62-1, 152st",기타,152st,152st(에스티),서울특별시,강남구,"테헤란로81길 62-1,",서울특별시 강남구 삼성동 152-2,"서울특별시 강남구 테헤란로81길 62-1, 지상2층 (삼성동)",204861.280358,445389.397758,가능,영업/정상
5,개포로20길 16 다미,한식,다미,다미,서울특별시,강남구,개포로20길 16,서울특별시 강남구 개포동 1232-2번지 호재빌딩1층,"서울특별시 강남구 개포로20길 16 (개포동,호재빌딩1층)",204129.096501,441728.951797,가능,영업/정상
6,"봉은사로73길 19, 한국피자헛",경양식,한국피자헛,한국피자헛 삼성중앙점,서울특별시,강남구,"봉은사로73길 19,",서울특별시 강남구 삼성동 48-4번지,"서울특별시 강남구 봉은사로73길 19, 지상1층 102호 (삼성동)",204379.015000,445712.725000,가능,영업/정상
7,"삼성로70길 6, 코너스톤디",기타,코너스톤디,코너스톤디,서울특별시,강남구,"삼성로70길 6,",서울특별시 강남구 대치동 962-13,"서울특별시 강남구 삼성로70길 6, 지상1층 (대치동)",205226.273805,444431.508768,가능,영업/정상
8,봉은사로2길 16 목구멍,일식,목구멍,목구멍 신논현점,서울특별시,강남구,봉은사로2길 16,서울특별시 강남구 역삼동 809-6,서울특별시 강남구 봉은사로2길 16 (역삼동),202197.669660,444601.630141,가능,영업/정상
9,"봉은사로 504, 길목 신관",한식,길목 신관,길목 신관,서울특별시,강남구,"봉은사로 504,","서울특별시 강남구 삼성동 145-2 지상1,2층","서울특별시 강남구 봉은사로 504, 지상1,2층 (삼성동)",204715.649628,445633.285715,가능,영업/정상


In [6]:
# 일단 검색결과 안나오고 바로 페이지가 넘어간다고 가정하고 

for i in range(0,len(df_test)):
    # 검색어를 네이버 url에 포함시켜 이동 + 검색 
    
    name = df_test['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

        
    time.sleep(3)  

    ## 화면이 안넘어가고 검색결과에 머물러 있을 때는 entryIframe 이 없음
    ## 그래서 검색어 쳤을때 entryIframe 있으면 --> 화면이 넘어갔다는 뜻이고 
    ## 없으면 아직 안넘어간 뜻이니깐 searchIframe 에서 클릭해서 넘어가야함 .
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
            
            time.sleep(3) 
            # 가게 이름, 변수로 지정
            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text
            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"
            
            # 리뷰 긁어오기 
            # PXMot 클래스 요소들을 모두 찾음
            pxmot_elements = driver.find_elements(By.CLASS_NAME, 'PXMot')
            

            # PXMot 가 존재할 때
            if pxmot_elements :
                for element in pxmot_elements :
                    element_text = element.text
                    if element_text.startswith('별점') : # 별점이 포함된 텍스트를 찾음
                        # 별점 텍스트가 있는 경우, 별점을 나타내므로 다음요소 검사
                        continue
                    else : 
                        # '별점' 텍스트가 없으면 방문자리뷰로 간주하여 값을 가져옴
                        review_text = element_text.replace('방문자리뷰 ','')

                        try :
                            review = review_text.replace(',','')
                            review = int(review)
                        except :
                            review = int(review)
                        break
            else  : 
                review = 0      
            
        else : 
            # searchIframe 프레임 

                searchIframe = driver.find_element(By.ID,'searchIframe')
                driver.switch_to.frame(searchIframe)


                ## 클릭 
                driver.find_element(By.CLASS_NAME,'YwYLL').click()
                
                # 기본프레임 
                time.sleep(1)
                driver.switch_to.default_content()
            

                # entryIframe 프레임 
                entryIframe = driver.find_element(By.ID,'entryIframe')
                driver.switch_to.frame(entryIframe)

                time.sleep(3) 
                # 가게 이름, 변수로 지정
                restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
                category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
                address = driver.find_element(By.CLASS_NAME,'LDgIH').text
                if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                    menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                    price = driver.find_element(By.CLASS_NAME,'mkBm3').text
                elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                    menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                    price = driver.find_element(By.CLASS_NAME,'Yrsei').text
                else : 
                    menu = '메뉴없음'
                    price = "가격없음"
                # if driver.find_elements(By.CLASS_NAME,'PXMot') :
                #     review = driver.find_element(By.CLASS_NAME,'PXMot').text
                #     review = int(review.replace('방문자리뷰 ',''))
                # else : 
                #     review = 0
                # 리뷰 긁어오기 
            # PXMot 클래스 요소들을 모두 찾음
                pxmot_elements = driver.find_elements(By.CLASS_NAME, 'PXMot')
            

            # PXMot 가 존재할 때
                if pxmot_elements :
                    for element in pxmot_elements :
                        element_text = element.text
                        if element_text.startswith('별점') : # 별점이 포함된 텍스트를 찾음
                            # 별점 텍스트가 있는 경우, 별점을 나타내므로 다음요소 검사
                            continue
                        else : 
                            # '별점' 텍스트가 없으면 방문자리뷰로 간주하여 값을 가져옴
                            review_text = element_text.replace('방문자리뷰 ','')

                            try :
                                review = review_text.replace(',','')
                                review = int(review)
                            except :
                                review = int(review)
                            break
                else  : 
                    review = 0      


    except Exception as e : 
        restaurant_name = name 
        category_name = '전처리필요'
        address = '전처리필요'
        menu = '메뉴없음'
        price = "가격없음"
        review = 0
        print(f"에러 메시지: {str(e)}")


    ## 리스트 ㄱ 
    restaurant_name_list.append(restaurant_name)
    category_name_list.append(category_name)
    address_list.append(address)
    menu_list.append(menu)
    price_list.append(price)
    review_list.append(review)

    
    print('식당 이름:',restaurant_name)
    print('식당 카테고리:',category_name)
    print('주소:',address)
    print('대표 메뉴:',menu)
    print('가격:',price)
    print('리뷰 : ',review)
    print('-----------------------------')
df_test_result = pd.DataFrame({'식당이름':restaurant_name_list ,
                             '업태구분' :category_name_list ,
                             '주소' :address_list ,
                             '메뉴': menu_list,
                             '가격':price_list,
                             '리뷰':review_list
                             })

    

식당 이름: 남오토코 역삼점
식당 카테고리: 이자카야
주소: 서울 강남구 논현로85길 29 2층
대표 메뉴: 꼬치8종
가격: 35,500원
리뷰 :  300
-----------------------------
식당 이름: 뢰벤돈까스
식당 카테고리: 양식
주소: 서울 강남구 테헤란로 124 삼원타워 지하1층 B102호
대표 메뉴: 등심돈까스
가격: 9,000원
리뷰 :  301
-----------------------------
식당 이름: 생강김밥 본점
식당 카테고리: 김밥
주소: 서울 강남구 남부순환로 2738 상일빌딩
대표 메뉴: 문어김밥
가격: 4,800원
리뷰 :  2028
-----------------------------
식당 이름: 삼성원조양곱창
식당 카테고리: 곱창,막창,양
주소: 서울 강남구 학동로101길 7 승창빌딩 1층
대표 메뉴: 특양
가격: 30,000원
리뷰 :  314
-----------------------------
식당 이름: 152st whisky bar
식당 카테고리: 바(BAR)
주소: 서울 강남구 테헤란로81길 62-1 2층 152st whisky bar
대표 메뉴: 보테가골드+치즈플레이트
가격: 300,000원
리뷰 :  1
-----------------------------
식당 이름: 다미가정식백반
식당 카테고리: 백반,가정식
주소: 서울 강남구 개포로20길 16
대표 메뉴: 가정식백반
가격: 9,000원
리뷰 :  53
-----------------------------
식당 이름: 피자헛 삼성중앙점
식당 카테고리: 피자
주소: 서울 강남구 봉은사로73길 19
대표 메뉴: 핫소스(2.5g)
가격: 100원
리뷰 :  392
-----------------------------
식당 이름: 코너스톤디
식당 카테고리: 카페,디저트
주소: 서울 강남구 삼성로70길 6 1층
대표 메뉴: New. 블루라떼
가격: 5,800원
리뷰 :  13
-----------------------------
식당 이름: 목구멍

In [7]:
df_test_result.head()


,식당이름,업태구분,주소,메뉴,가격,리뷰
0,남오토코 역삼점,이자카야,서울 강남구 논현로85길 29 2층,꼬치8종,"35,500원",300
1,뢰벤돈까스,양식,서울 강남구 테헤란로 124 삼원타워 지하1층 B102호,등심돈까스,"9,000원",301
2,생강김밥 본점,김밥,서울 강남구 남부순환로 2738 상일빌딩,문어김밥,"4,800원",2028
3,삼성원조양곱창,"곱창,막창,양",서울 강남구 학동로101길 7 승창빌딩 1층,특양,"30,000원",314
4,152st whisky bar,바(BAR),서울 강남구 테헤란로81길 62-1 2층 152st whisky bar,보테가골드+치즈플레이트,"300,000원",1


In [8]:
df_test_result['업태구분'].unique()

array(['이자카야', '양식', '김밥', '곱창,막창,양', '바(BAR)', '백반,가정식', '피자', '카페,디저트',
       '육류,고기요리', '돼지고기구이', '요리주점', '한식뷔페', '뷔페', '한식', '돈가스', '장소대여',
       '일식당', '카페', '순대,순댓국', 'PC방', '바나프레소', '예식장', '주류'], dtype=object)

In [12]:
# 필요없는 카테고리(카페, 술, 출장요리 등등) 제거
제외 = ['바나프레소','이자카야','바(BAR)','카페,디저트','요리주점','장소대여','카페','PC','예식장']
for i in 제외 :
    df_test_result = df_test_result.drop(df_test_result[df_test_result['업태구분'] == i].index)

len(df_test_result)

18

In [13]:
df_test_result

,식당이름,업태구분,주소,메뉴,가격,리뷰
1,뢰벤돈까스,양식,서울 강남구 테헤란로 124 삼원타워 지하1층 B102호,등심돈까스,"9,000원",301
2,생강김밥 본점,김밥,서울 강남구 남부순환로 2738 상일빌딩,문어김밥,"4,800원",2028
3,삼성원조양곱창,"곱창,막창,양",서울 강남구 학동로101길 7 승창빌딩 1층,특양,"30,000원",314
5,다미가정식백반,"백반,가정식",서울 강남구 개포로20길 16,가정식백반,"9,000원",53
6,피자헛 삼성중앙점,피자,서울 강남구 봉은사로73길 19,핫소스(2.5g),100원,392
8,목구멍 신논현점,"육류,고기요리",서울 강남구 봉은사로2길 16,미박삼겹살(150g),"13,000원",1137
9,길목 신관,돼지고기구이,서울 강남구 봉은사로 504 진양빌딩 1층,투뿔목살,"18,000원",54
11,집밥푸드,한식뷔페,서울 강남구 선릉로 28,메뉴없음,가격없음,3
12,더 키푸드,뷔페,서울 강남구 선릉로 823 6층,정찬,"10,000원",87
13,정육점 아저씨의 투잡 대치지점,한식,서울 강남구 삼성로81길 43,메뉴없음,가격없음,1


In [ ]:
df_test_result.to_csv('매장정보결과_test.csv')

,식당이름,업태구분,주소,메뉴,가격
0,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원"
1,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원"
2,채선당 샤브보트 강남역점,샤브샤브,서울 강남구 강남대로 378 . 지상1층,소고기 샤브샤브,"12,900원"
3,수아당 갤러리아백화점 명품관 WEST점,김밥,서울 강남구 압구정로 343,유부월남쌈,"18,000원"
4,알찬한끼,종합분식,서울 강남구 역삼로 204 삼환아르누보 지하1층 107호,알찬도시락,"5,000원"
5,세계관 삼성본점,한식,서울 강남구 삼성로96길 11 2층,투뿔 한우 곱도리탕,"32,000원"
6,분식을품다 한티점,종합분식,서울 강남구 역삼로 409 1층 102호,쌀 떡볶이,"5,900원"
7,식당민홍,일식당,서울 강남구 논현로146길 11 2층,오마카세 코스,"122,000원"
8,제주그집,한식,서울 강남구 논현로8길 11 지상1층,냉동삼겹살(150g),"14,000원"
9,만수정 강남점,"장어,먹장어요리",서울 강남구 강남대로 628 비율라빌딩 1층,민물장어 1kg ( 650~680g),"90,000원"


# 여기서 부터 찐 코드 , 위에건 그냥 테스트 -> 되면 바로 지우겠습니다

In [6]:
# 일단 검색결과 안나오고 바로 페이지가 넘어간다고 가정하고 

for i in range(0,len(res_df)):
    # 검색어를 네이버 url에 포함시켜 이동 + 검색 
    
    name = res_df['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

        
    time.sleep(3)  

    ## 화면이 안넘어가고 검색결과에 머물러 있을 때는 entryIframe 이 없음
    ## 그래서 검색어 쳤을때 entryIframe 있으면 --> 화면이 넘어갔다는 뜻이고 
    ## 없으면 아직 안넘어간 뜻이니깐 searchIframe 에서 클릭해서 넘어가야함 .
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
            
            time.sleep(3) 
            # 가게 이름, 변수로 지정
            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text
            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"
            
            # 있으면 불러오고 없으면 없다고 ㄱㄱ
            if driver.find_elements(By.CLASS_NAME,'PXMot') :
                review = driver.find_element(By.CLASS_NAME,'PXMot').text
                review = int(review.replace('방문자리뷰 ',''))
            else : 
                review=0
            
        else : 
            # searchIframe 프레임 

                searchIframe = driver.find_element(By.ID,'searchIframe')
                driver.switch_to.frame(searchIframe)


                ## 클릭 
                driver.find_element(By.CLASS_NAME,'YwYLL').click()
                
                # 기본프레임 
                time.sleep(1)
                driver.switch_to.default_content()
            

                # entryIframe 프레임 
                entryIframe = driver.find_element(By.ID,'entryIframe')
                driver.switch_to.frame(entryIframe)

                time.sleep(3) 
                # 가게 이름, 변수로 지정
                restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
                category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
                address = driver.find_element(By.CLASS_NAME,'LDgIH').text
                if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                    menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                    price = driver.find_element(By.CLASS_NAME,'mkBm3').text
                elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                    menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                    price = driver.find_element(By.CLASS_NAME,'Yrsei').text
                else : 
                    menu = '메뉴없음'
                    price = "가격없음"
                if driver.find_elements(By.CLASS_NAME,'PXMot') :
                    review = driver.find_element(By.CLASS_NAME,'PXMot').text
                    review = int(review.replace('방문자리뷰 ',''))
                else : 
                    review = 0



    except Exception as e : 
        restaurant_name = name 
        category_name = '전처리필요'
        address = '전처리필요'
        menu = '메뉴없음'
        price = "가격없음"
        review = 0
        print(f"에러 메시지: {str(e)}")


    ## 리스트 ㄱ 
    restaurant_name_list.append(restaurant_name)
    category_name_list.append(category_name)
    address_list.append(address)
    menu_list.append(menu)
    price_list.append(price)
    review_list.append(review)

    
    print('식당 이름:',restaurant_name)
    print('식당 카테고리:',category_name)
    print('주소:',address)
    print('대표 메뉴:',menu)
    print('가격:',price)
    print('리뷰 : ',review)
    print('-----------------------------')
res_info_df = pd.DataFrame({'식당이름':restaurant_name_list ,
                             '업태구분' :category_name_list ,
                             '주소' :address_list ,
                             '메뉴': menu_list,
                             '가격':price_list,
                             '리뷰':review_list
                             })

    

식당 이름: 정식당
식당 카테고리: 한식
주소: 서울 강남구 강남대로114길 18 . 1층 정식당
대표 메뉴: 소불고기주물럭
가격: 16,000원
리뷰 :  9
-----------------------------
식당 이름: 채선당 샤브보트 강남역점
식당 카테고리: 샤브샤브
주소: 서울 강남구 강남대로 378 . 지상1층
대표 메뉴: 소고기 샤브샤브
가격: 12,900원
리뷰 :  71
-----------------------------
식당 이름: 수아당 갤러리아백화점 명품관 WEST점
식당 카테고리: 김밥
주소: 서울 강남구 압구정로 343
대표 메뉴: 비빔면
가격: 10,000원
리뷰 :  9
-----------------------------
식당 이름: 알찬한끼
식당 카테고리: 종합분식
주소: 서울 강남구 역삼로 204 삼환아르누보 지하1층 107호
대표 메뉴: 알찬도시락
가격: 5,000원
리뷰 :  0
-----------------------------
식당 이름: 세계관 삼성본점
식당 카테고리: 한식
주소: 서울 강남구 삼성로96길 11 2층
대표 메뉴: 투뿔 한우 곱도리탕
가격: 32,000원
리뷰 :  59
-----------------------------
식당 이름: 분식을품다 한티점
식당 카테고리: 종합분식
주소: 서울 강남구 역삼로 409 1층 102호
대표 메뉴: 쌀 떡볶이
가격: 5,900원
리뷰 :  28
-----------------------------


['정식당', '채선당 샤브보트 강남역점', '수아당', '알찬한끼', '세계관', '분식을 품다', '식당 민홍',
       '제주그집', '만수정 강남점', '은서네 국내산 대패삼겹살', '냉동고집 삼성점', '리하베스트', '팡뮤제',
       '만소당', '꼬모 서울', '겐지(KENJI)', '논현136', '피양옥', "세븐어클락(7O'CLOCK)",

In [7]:
review_list

[9, 71, 9, 0, 59, 28]

In [8]:
res_info_df.head()


,식당이름,업태구분,주소,메뉴,가격,리뷰
0,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원",9
1,채선당 샤브보트 강남역점,샤브샤브,서울 강남구 강남대로 378 . 지상1층,소고기 샤브샤브,"12,900원",71
2,수아당 갤러리아백화점 명품관 WEST점,김밥,서울 강남구 압구정로 343,비빔면,"10,000원",9
3,알찬한끼,종합분식,서울 강남구 역삼로 204 삼환아르누보 지하1층 107호,알찬도시락,"5,000원",0
4,세계관 삼성본점,한식,서울 강남구 삼성로96길 11 2층,투뿔 한우 곱도리탕,"32,000원",59


In [ ]:
res_info_df.to_csv('매장정보결과.csv')

,식당이름,업태구분,주소,메뉴,가격
0,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원"
1,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원"
2,채선당 샤브보트 강남역점,샤브샤브,서울 강남구 강남대로 378 . 지상1층,소고기 샤브샤브,"12,900원"
3,수아당 갤러리아백화점 명품관 WEST점,김밥,서울 강남구 압구정로 343,유부월남쌈,"18,000원"
4,알찬한끼,종합분식,서울 강남구 역삼로 204 삼환아르누보 지하1층 107호,알찬도시락,"5,000원"
5,세계관 삼성본점,한식,서울 강남구 삼성로96길 11 2층,투뿔 한우 곱도리탕,"32,000원"
6,분식을품다 한티점,종합분식,서울 강남구 역삼로 409 1층 102호,쌀 떡볶이,"5,900원"
7,식당민홍,일식당,서울 강남구 논현로146길 11 2층,오마카세 코스,"122,000원"
8,제주그집,한식,서울 강남구 논현로8길 11 지상1층,냉동삼겹살(150g),"14,000원"
9,만수정 강남점,"장어,먹장어요리",서울 강남구 강남대로 628 비율라빌딩 1층,민물장어 1kg ( 650~680g),"90,000원"
